# Introduction

In this notebook, the estimates for the energy consumption of the top 100 mineable cryptocurrencies is calculated. 
In the first section, a hardware based estimation is made. Secondly, a revenue based is made. 
After this, we take a look at the electronic waste by mining. Finally, we investigate the results and check for inter-and extrapollation.

In [1]:
# Import libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from ipywidgets import *
from datetime import datetime
from IPython.display import Image
from matplotlib.dates import DateFormatter

In [2]:
DATA_PATH = 'data/'
BTC_PATH = DATA_PATH + 'bitcoin/'
HARDWARE_PATH = DATA_PATH+'hardware/'

## Parameters

In [6]:
seconds_in_year = 31536000
hours_in_year = 8760
kWh_to_J = 3600*10**3
W_to_TWh = hours_in_year/(10**12)

std_PUE = {'lower': 1, 'estimate': 1.1, 'upper':1.2}
revenue_percent = 0.60

# Plots
date_form = DateFormatter("%m-%d")

## Geography 

### Coinshares

In [ ]:
# US: https://www.statista.com/statistics/190680/us-industrial-consumer-price-estimates-for-retail-electricity-since-1970/
# China: https://www.mondaq.com/china/oil-gas-electricity/804336/china-electricity-prices-for-industrial-consumers
# Russia: https://www.mondaq.com/china/oil-gas-electricity/804336/china-electricity-prices-for-industrial-consumers
# Kazakhstan: https://www.globalpetrolprices.com/Kazakhstan/electricity_prices/
# Malaysia: https://www.globalpetrolprices.com/Malaysia/electricity_prices/
# Iran: https://financialtribune.com/articles/energy/97278/rise-in-electricity-and-water-tariffs-in-iran

In [9]:
data = {'location': ['China', 'US', 'Russia', 'Kazakhstan', 'Malaysia','Iran','Canada','Norway','Germany'],
        'share': [66.17, 7.15, 6.43, 5.89, 4.22, 3.67, 0.80, 0.67, 0.5],
        'normal_price': [0.074, 0.0693, 0.041, 0.046, 0.101, 0.07, 0.10, 0.08, 0.2]}
CBECI_geography = pd.DataFrame(data)

In [6]:
china_data = {'location': ['Xinjiang', 'Sichuan', 'Nei Mongol', 'Yunnan', 'Beijing', 'Zhejiang', 'Shanxi', 'Gansu', 'Guizhou', 'Qinhai'],
             'share': [36.49,9.74,8.22,5.5,1.5,0.82,0.71,0.57,0.28,0.26]}
CBECI_geography_china = CBECI_geography.append(pd.DataFrame(china_data)).set_index('location').drop('China').sort_values(by='share', ascending=False)

In [ ]:
data = {'location': ['Sichuan', 'China', 'Non-China', 'Other'],
        'share': [0.54,0.11,0.31,0.04],
        'normal_price': [0.074, 0.084, 0.061, 0.084], # World average for non-china
        'min_price': [0.035, 0.084, 0.061, 0.084], # American price for non-china
        'renewable': [0,0,0,0]}